In [2]:
from minirl.agents.pg import PGAgent

from mlopskit import make

model_db = make("cache/feature_store-v1",db_name="lmodel.db")

2023-05-09 10:07:22 [info     ] APIs of mlopskit               model_name=feature_store model_version=1 ops_type=cache
2023-05-09 10:07:22 [info     ] The list of all versions for the current model is [1, 2].
2023-05-09 10:07:22 [warning  ] The version 1 is out of date. You should consider upgrading to version `v2`.
2023-05-09 10:07:22 [info     ] Usage of mlopskit-cache        Params={'db_type': 'rlite/redis/sfdb/diskcache, default:rlite', 'return_type': 'dblink/dbobj, default: dbobj', 'db_name': 'default: rlite_model.cache'}


In [3]:
model_db.rpush("rpush","1")

2

In [4]:
model_db.lpush("rpush","3")

3

In [5]:
model_db.lrange("rpush","0","2")


['3', '3', '1']

In [31]:
model_db.delete("rpush")

1

In [ ]:
model_db.lrem("rpush","2")

: 

: 

In [16]:
model_db.llen("rpush2")


0

In [5]:
%%file pg_newmodel.py

from itertools import count
import gym
import numpy as np
from minirl.agents.pg import PGAgent

from mlopskit import make

model_db = make("cache/feature_store-v1",db_name="lmodel.db")
model_id="pg"
env = gym.make('CartPole-v1')
ob_n = env.observation_space.shape[0]
ac_n = env.action_space.n
agent=PGAgent(ob_n,ac_n,model_db=model_db,model_id=model_id)
render_interval = -1
log_interval=100
def main():
    """Run REINFORCE algorithm to train on the environment"""
    avg_reward = []
    for i_episode in count(1):
        ep_reward = 0
        obs,_ = env.reset()
        for t in range(100):  # Don't infinite loop while learning
            
            action = agent.act(obs,model_id)
            next_obs, reward, done, _,_ = env.step(action)
            ep_reward += reward
            agent.rewards.append(reward)

            if render_interval != -1 and i_episode % render_interval == 0:
                env.render()

            if done:
                break
                
            obs=next_obs

        agent.learn(model_id)

        if i_episode % log_interval == 0:
            print("Ave reward: {}".format(sum(avg_reward)/len(avg_reward)))
            avg_reward = []

        else:
            avg_reward.append(ep_reward)


main()

Overwriting pg_newmodel.py
